<a href="https://colab.research.google.com/github/resloved/RWKV-notebooks/blob/master/RWKV_v4_RNN_Pile_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RWKV-v4-RNN-Pile Fine-Tuning

[RWKV](https://github.com/BlinkDL/RWKV-LM) is an RNN with transformer-level performance


This notebook aims to streamline fine-tuning RWKV-v4 models


## Setup

In [1]:
#@title Google Drive Options { display-mode: "form" }
save_models_to_drive = True #@param {type:"boolean"}
drive_mount = '/content/drive' #@param {type:"string"}
output_dir = 'rwkv-v4-rnn-pile-tuning' #@param {type:"string"}
tuned_model_name = 'tuned' #@param {type:"string"}

import os
from google.colab import drive
if save_models_to_drive:
    from google.colab import drive
    drive.mount(drive_mount)
    
output_path = f"{drive_mount}/MyDrive/{output_dir}" if save_models_to_drive else f"/content/{output_dir}"
os.makedirs(f"{output_path}/{tuned_model_name}", exist_ok=True)
os.makedirs(f"{output_path}/base_models/", exist_ok=True)

print(f"Saving models to {output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saving models to /content/drive/MyDrive/rwkv-v4-rnn-pile-tuning


In [2]:
!nvidia-smi

Fri Sep  2 15:16:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    36W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!git clone https://github.com/blinkdl/RWKV-LM
repo_dir = "/content/RWKV-LM/RWKV-v4"
%cd $repo_dir

fatal: destination path 'RWKV-LM' already exists and is not an empty directory.
/content/RWKV-LM/RWKV-v4


In [4]:
!pip install transformers pytorch-lightning deepspeed wandb ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Load Base Model




In [5]:
#@title Base Model Options
#@markdown Using any of the listed options will download the checkpoint from huggingface

base_model_name = "RWKV-4-Pile-430M" #@param ["RWKV-4-Pile-1B5", "RWKV-4-Pile-430M", "RWKV-4-Pile-169M"]
base_model_url = f"https://huggingface.co/BlinkDL/{base_model_name.lower()}"

# This may take a while
!git lfs clone $base_model_url

from glob import glob
base_model_path = glob(f"{base_model_name.lower()}/RWKV*.pth")[0]

print(f"Using {base_model_path} as base")

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
fatal: destination path 'rwkv-4-pile-430m' already exists and is not an empty directory.
Error(s) during clone:
git clone failed: exit status 128
Using rwkv-4-pile-430m/RWKV-4-Pile-430M-20220808-8066.pth as base


## Generate Training Data

In [6]:
#@title Training Data Options
#@markdown `input_file` should be the path to a single file that contains the text you want to fine-tune with.
#@markdown Either upload a file to this notebook instance or reference a file in your Google drive.

import numpy as np
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(tokenizer_file=f'{repo_dir}/20B_tokenizer.json')

input_file = "/content/drive/MyDrive/training.txt" #@param {type:"string"}
output_file = 'train.npy'

print(f'Tokenizing {input_file} (VERY slow. please wait)')

data_raw = open(input_file, encoding="utf-8").read()
print(f'Raw length = {len(data_raw)}')

data_code = tokenizer.encode(data_raw)
print(f'Tokenized length = {len(data_code)}')

out = np.array(data_code, dtype='uint16')
np.save(output_file, out, allow_pickle=False)

Tokenizing /content/drive/MyDrive/conversation-sentiment.txt (VERY slow. please wait)
Raw length = 23533698
Tokenized length = 5948757


## Training

In [15]:
#@title Training Options { display-mode: "form" }
from shutil import copy
import os

def training_options():
    EXPRESS_PILE_MODE = True
    EXPRESS_PILE_MODEL_NAME = base_model_path.split(".")[0]
    EXPRESS_PILE_MODEL_TYPE = base_model_name
    n_epoch = 100 #@param {type:"integer"}
    epoch_save_frequency = 25 #@param {type:"integer"}
    batch_size =  7#@param {type:"integer"} 
    ctx_len = 384 #@param {type:"integer"}
    epoch_save_path = f"{output_path}/{tuned_model_name}"
    return locals()

def model_options():
    T_MAX = 384 #@param {type:"integer"}
    return locals()

def env_vars():
    RWKV_FLOAT_MODE = 'fp16' #@param ['fp16', 'bf16', 'bf32'] {type:"string"}
    RWKV_DEEP_SPEED = '1' #@param ['0', '1'] {type:"string"}
    return {f"os.environ['{key}']": value for key, value in locals().items()}

def replace_lines(file_name, to_replace):
    with open(file_name, 'r') as f:
        lines = f.readlines()
    with open(f'{file_name}.tmp', 'w') as f:
        for line in lines:
            key = line.split(" =")[0]
            if key.strip() in to_replace:
                value = to_replace[key.strip()]
                if isinstance(value, str):
                    f.write(f'{key} = "{value}"\n')
                else:
                    f.write(f'{key} = {value}\n')
            else:
                f.write(line)
    copy(f'{file_name}.tmp', file_name)
    os.remove(f'{file_name}.tmp')

values = training_options()
values.update(env_vars())
replace_lines('train.py', values)
replace_lines('src/model.py', model_options())

In [16]:
!python train.py 

loading numpy data... train.npy
current vocab size = 50277 (make sure it's correct)
data has 5948757 tokens.
2022-09-02 15:22:19 - INFO - torch.distributed.nn.jit.instantiator - Created a temporary directory at /tmp/tmpo_1mmdlu
2022-09-02 15:22:19 - INFO - torch.distributed.nn.jit.instantiator - Writing /tmp/tmpo_1mmdlu/_remote_module_non_scriptable.py
2022-09-02 15:22:23 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.

RWKV_HEAD_QK_DIM 0

Using /root/.cache/torch_extensions/py37_cu113 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py37_cu113/wkv/build.ninja...
Building extension module wkv...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv...

model RWKV fp16 epoch 100 batchsz 7 betas (0.9, 0.999) eps 1e-08 ctx 384 layer 24 embd 1024 

2022-09-02 15:22:25 - INFO - pytorch_lightning.ut